In [1]:
import requests
import re
from collections import OrderedDict
from itertools import repeat
from tqdm import tqdm_notebook
import pandas as pd
import lxml.html
from selenium.webdriver import Chrome

In [3]:
data = pd.read_csv('poetry_url.csv')

In [56]:
data.head()

,url,title,writer
0,http://poemlove.co.kr/m/bbs/board.php?bo_table...,동시/첫눈 오는 날.,장수남
1,http://poemlove.co.kr/m/bbs/board.php?bo_table...,"그리움, 너는",고은영
2,http://poemlove.co.kr/m/bbs/board.php?bo_table...,비워내는 건 외로운 법이다,고은영
3,http://poemlove.co.kr/m/bbs/board.php?bo_table...,지극히 따뜻한 애혼(愛魂)에게,고은영
4,http://poemlove.co.kr/m/bbs/board.php?bo_table...,12월엔 설화(雪花)를 그리는 한 마리 노루이고 싶다,고은영


In [5]:
urls = data.url

In [33]:
urls[10]

'http://poemlove.co.kr/m/bbs/board.php?bo_table=tb01&wr_id=254754'

In [74]:
contents = []
for i in tqdm_notebook(urls):
    try:
        req = requests.get(i)
        root = lxml.html.fromstring(req.text)
        text = root.cssselect('span#writeContents')[0].text_content()
        text = re.sub("[(\xa0)]", "", text)
        text = re.sub('[(\u3000)]',' ', text)
        text = re.sub("[a-zA-Z]+ *?[a-zA-Z]+", "<ENG>", text)
        text = re.sub("[0-9]+ *?[0-9]+", "<NUM>", text)
        text = re.sub("[一-龥]+", "<CHI>", text)
        text = re.sub("[ぁ-ゔ]+|[ァ-ヴー]+", "<JAP>", text)
        contents.append([x.strip() for x in re.split(r"[\r]", text) if len(x.strip()) > 1])
    except:
        contents.append(None)

In [75]:
data['contents'] = contents

In [76]:
data.to_csv('poetry.csv',encoding='UTF8',index=False)

In [2]:
poetry = pd.read_csv('poetry.csv')

In [3]:
author = pd.read_csv('poetry_author.csv')

In [21]:
poetry['writer'] = poetry.writer.apply(lambda x: 'poemlove' if x=='none' else x)

In [10]:
author = author[52:-25]
author = author.reset_index(drop = True)

In [47]:
poetry = poetry.drop_duplicates(['url'],keep='first').reset_index(drop = True)

In [133]:
poetry_final = pd.concat([poetry,author],axis=1)
poetry_final = poetry_final.iloc[:,[0,1,2,3,5]]

In [134]:
poetry_final = poetry_final.drop_duplicates(['contents'],keep='first').reset_index(drop = True)
len(poetry_final)

143736

In [135]:
poetry_final[poetry_final.author.isnull() == True]

,url,title,writer,contents,author
2053,http://poemlove.co.kr/m/bbs/board.php?bo_table...,특집/셸리 탄생 200주년 기념 특집 - 셸리(Percy Bysshe Shelly)...,poemlove,"['‘시문학’ 지 <NUM>년 7월호:', '특집/셸리 탄생 <NUM>주년 기념 특...",NaN
12266,http://poemlove.co.kr/m/bbs/board.php?bo_table...,대작(對酌),김안로,"['대작<CHI>/김안로', '나는 막걸리를 들고', '하늘은 구름을 들이켰지', ...",NaN
15614,http://poemlove.co.kr/m/bbs/board.php?bo_table...,"해운대 찬가(海雲臺, 讚歌)/李時明",이시명,"['해운대 찬가<CHI>, <CHI>', '/李<CHI>', '국유 현묘지도와 ""천...",NaN
18130,http://poemlove.co.kr/m/bbs/board.php?bo_table...,겨울 그리고 봄,송정숙,"['겨울 그리고 봄', '-송정숙-', '밑그림을 그린다', '너를 위해서', '꽃...",NaN
18132,http://poemlove.co.kr/m/bbs/board.php?bo_table...,창 밖에는 눈3,송정숙,"['창 밖에는 눈3', '-송정숙-', '오전에는 봄', '오후에는 겨울 눈', '...",NaN
18133,http://poemlove.co.kr/m/bbs/board.php?bo_table...,창 밖에는 눈2,송정숙,"['창 밖에는 눈2', '어찌하나요', '떠난다고 하는데', '이렇게 웃죠', '마...",NaN
18134,http://poemlove.co.kr/m/bbs/board.php?bo_table...,창 밖에는 눈1,송정숙,"['창 밖에는 눈1', '-송정숙-', '햇살 뒷 걸음', '마지막 꽃이 되자', ...",NaN
18199,http://poemlove.co.kr/m/bbs/board.php?bo_table...,구름에 숨은 달,송정숙,"['구름에 숨은 달', '-송정숙-', '직무유기다', '법원서 출두하래', '보름...",NaN
18200,http://poemlove.co.kr/m/bbs/board.php?bo_table...,빨간 신호등으로 멈추지 못 한다,송정숙,"['빨간 신호등으로 멈추지 못 한다', '-송정숙-', '날개가 있어야', '날아 ...",NaN
18201,http://poemlove.co.kr/m/bbs/board.php?bo_table...,그믐달,송정숙,"['그믐달', '-송정숙-', '주고 또 준다', '마지막은 자존심', '지키게 해...",NaN


In [136]:
poetry_final.isnull().sum()

url           0
title        13
writer        0
contents      1
author      111
dtype: int64

In [137]:
poetry_final = poetry_final[poetry_final.title.isnull() == False].reset_index(drop = True)
poetry_final = poetry_final[poetry_final.contents.isnull() == False].reset_index(drop = True)
poetry_final = poetry_final[poetry_final.author.isnull() == False].reset_index(drop = True)

In [138]:
poetry_final['title'] = poetry_final.title.apply(lambda x: x.split(' - '))

In [139]:
poetry_final['title']

0                            [동시/첫눈 오는 날.]
1                                [그리움, 너는]
2                         [비워내는 건 외로운 법이다]
3                       [지극히 따뜻한 애혼(愛魂)에게]
4         [12월엔  설화(雪花)를 그리는 한 마리 노루이고 싶다]
                        ...               
143717                         [가죽나무, 도종환]
143718                       [당신의 나라, 서주홍]
143719                       [물먹는 하마, 강문숙]
143720             [혼자 일 수 밖에 없었던 이유, 이용채]
143721                          [한사람, 이정하]
Name: title, Length: 143722, dtype: object

In [140]:
a = []
for i in range(len(poetry_final.title)):
    if len(poetry_final.title[i]) > 2:
        a.append(i)

In [141]:
len(a)

453

In [142]:
poetry_final = poetry_final.drop(a,0).reset_index(drop = True)

In [149]:
poetry_final.isnull().sum()

url         0
title       0
writer      0
contents    0
author      0
dtype: int64

In [128]:
b = []
for i in range(len(poetry_final.title)):
    if len(poetry_final.title[i]) == 2:
        b.append([poetry_final.title[i][0]])
    else:
        b.append(poetry_final.title[i])

In [148]:
len(b)

143269